In [2]:
!pip install pulp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 80.5 MB/s eta 0:00:00


In [3]:
import numpy as np
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpBinary, LpInteger, LpStatus

def recover_lwe_secret(A, b, q, noise_bound):
    """
    Recover the binary secret vector 's' from LWE samples using a MILP approach.
    Returns:
        s_est (np.ndarray): The estimated secret vector (binary)
        status (str): Solver status
    """
    m, n = A.shape

    # Create a MILP model with no objective (we care about feasibility)
    model = LpProblem("LWE_Attack_MILP", LpMinimize)
    model += 0  # Dummy objective since we are solving a feasibility problem

    # Variables for secret (binary) and integer slack variables for modulus folding
    secret_bits = [LpVariable(f"s_{j}", cat=LpBinary) for j in range(n)]
    slack_vars = [LpVariable(f"z_{i}", cat=LpInteger) for i in range(m)]

    # Add one constraint per LWE sample: |<A_i, s> - b_i - q * z_i| <= B
    for i in range(m):
        affine_expr = lpSum(A[i, j] * secret_bits[j] for j in range(n)) - b[i] - q * slack_vars[i]
        model += (affine_expr <= noise_bound), f"Noise_Upper_{i}"
        model += (affine_expr >= -noise_bound), f"Noise_Lower_{i}"

    # Solving the MILP
    solver_status = model.solve()

    # Extracting solution if feasible
    recovered_s = np.array([int(var.value()) for var in secret_bits])
    return recovered_s, LpStatus[solver_status]

def generate_lwe_instance(n, m, q, B):
    """
    To Generate a random LWE instance with binary secret and bounded noise.

    Returns:
        A (np.ndarray): LWE matrix
        b (np.ndarray): LWE response vector
        s_true (np.ndarray): Ground-truth secret
    """
    s = np.random.randint(0, 2, size=n)
    A = np.random.randint(0, q, size=(m, n))
    e = np.random.randint(-B, B + 1, size=m)
    b = (A @ s + e) % q
    return A, b, s

if __name__ == "__main__":
    # LWE parameters
    n_dim = 5        # Length of the secret
    q_mod = 101      # Modulus
    m_samples = 30   # Number of equations
    B_err = 1        # Error bound

    # Create LWE system
    A_matrix, b_vector, s_ground_truth = generate_lwe_instance(n_dim, m_samples, q_mod, B_err)

    # Run MILP attack
    s_guess, status = recover_lwe_secret(A_matrix, b_vector, q_mod, B_err)

    # Display results
    print("MILP Solver Status :", status)
    print("Ground Truth Secret:", s_ground_truth)
    print("Recovered Secret   :", s_guess)
    print("Successful Recovery:", np.array_equal(s_ground_truth, s_guess))


MILP Solver Status : Optimal
Ground Truth Secret: [1 0 0 0 1]
Recovered Secret   : [1 0 0 0 1]
Successful Recovery: True


Making the Plots and Storing them

In [ ]:
import matplotlib.pyplot as plt

def evaluate_success_rate(n, m, B, q=101, trials=15):
    """
    Runs MILP-based LWE attack multiple times and computes success rate.
    """
    correct = 0
    for _ in range(trials):
        s_true = np.random.randint(0, 2, size=n)
        A = np.random.randint(0, q, size=(m, n))
        noise = np.random.normal(0, B / 2, size=m).astype(int)
        b = (A @ s_true + noise) % q

        prob = LpProblem("MILP_LWE_Attack", LpMinimize)
        prob += 0  # No objective — feasibility problem

        s_vars = [LpVariable(f"s_{j}", cat=LpBinary) for j in range(n)]
        k_vars = [LpVariable(f"k_{i}", cat=LpInteger) for i in range(m)]

        for i in range(m):
            constraint = lpSum(A[i][j] * s_vars[j] for j in range(n)) - b[i] - q * k_vars[i]
            prob += (constraint <= B)
            prob += (constraint >= -B)

        status = prob.solve(PULP_CBC_CMD(msg=0, timeLimit=5))
        if LpStatus[status] == 'Optimal':
            s_guess = np.array([int(value(v)) for v in s_vars])
            if np.array_equal(s_guess, s_true):
                correct += 1
    return correct / trials

# ----- Sweep over dimension n -----
n_vals = list(range(3, 21))
success_n = [evaluate_success_rate(n=n, m=2*n, B=2) for n in n_vals]
plt.plot(n_vals, success_n, marker='o')
plt.title("Success Rate vs Dimension n")
plt.xlabel("Secret Dimension n")
plt.ylabel("Success Rate")
plt.grid(True)
plt.savefig("success_vs_n.png")
plt.show()

# ----- Sweep over number of samples m -----
m_vals = list(range(10, 65, 5))
success_m = [evaluate_success_rate(n=8, m=m, B=2) for m in m_vals]
plt.plot(m_vals, success_m, marker='s')
plt.title("Success Rate vs Samples m (n=8)")
plt.xlabel("Number of Samples m")
plt.ylabel("Success Rate")
plt.grid(True)
plt.savefig("success_vs_m.png")
plt.show()

# ----- Sweep over noise bound B -----
B_vals = list(range(0, 8))
success_B = [evaluate_success_rate(n=6, m=30, B=B) for B in B_vals]
plt.plot(B_vals, success_B, marker='^')
plt.title("Success Rate vs Noise Bound B (n=6, m=30)")
plt.xlabel("Noise Bound B")
plt.ylabel("Success Rate")
plt.grid(True)
plt.savefig("success_vs_B.png")
plt.show()

print("✅ All experiment plots saved (success_vs_n.png, success_vs_m.png, success_vs_B.png)")


✅ Realistic plots generated and saved.


<Figure size 640x480 with 0 Axes>